In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py

In [3]:
from robust_motifs.timing import Timer
from robust_motifs.counting import get_n_extended_simplices, get_bisimplices
from robust_motifs.custom_mp import prepare_shared_memory
from pathlib import Path
import numpy as np
import scipy.sparse as sp
import pandas as pd
import time

In [4]:
from robust_motifs.data import load_sparse_matrix_from_pkl, import_connectivity_matrix
from robust_motifs.plot import plot_matrices

In [5]:
from robust_motifs.simple_graphs import *
from pathlib import Path

In [6]:
import scipy.sparse as sp
import numpy as np
import h5py
import multiprocessing as mp
from itertools import product

In [7]:
funcs = [save_count_cyclic_extension, save_count_cyclic_extension_1_node, save_count_cyclic_extension_1_node, save_count_simplex_extension, save_count_circulant_extension_1_node, save_count_circulant_extension_1_node]

In [8]:
strings = ["Cyclic graph extension with n nodes", "Cyclic graph extension with 1 node", "Cyclic graph extension with 1 node", "Simplex extension with n nodes", "Circulant graph d. 2 extension with one node",  "Circulant graph d. 2 extension with one node"]

In [9]:
n_nodes = [50, 50, 100, 10, 10, 20]

In [10]:
pool = mp.Pool()

Given a directed graph $G$ with $n$ nodes and no bidirectional edges, we call the **extension** of $G$ the graph $G'$ obtained by adding $n$ nodes to $G$, such that node $n+j$ is bidirectionally connected to node $j$ and has no other edge. 

We call the **uninodal extension** of $G$ the graph $G''$ obtained by adding 1 node to G such that the extra node is bidirectionally connected to all other edges.

Let's call $S-G_{i}$ the number of simplices on $i$ nodes of $G$, and $ES-G_{i}$ the number of extended simplices on $i$ nodes on $G$. Then $S-G_{i}$ = $ES-G'_{i+1}$.

In particular, cyclic graphs and simplices have no bidirectional edges. So, for example, given a cyclic graph on $n$ nodes, its extension must have exactly $n$ extended simplices on 3 nodes.

I needed to make computations for $G''$ case by case. In particular, I checked that on a cyclic graph G on n nodes, $$ES-G''_{3} = n + n \cdot n-1 = n^2$$ $$ES-G''_{4} = n \cdot n-2$$ $$ES-G''_{k} = 0 \quad \mathrm{for} \quad k > 4$$ 

and on a circulant graph of order 2 on $n$ nodes: $$ES-G''_{3} = 2n + n \cdot (n-1)$$ $$ES-G''_{4} = n + 2n \cdot (n - 2)$$ $$ES-G''_{5} = n \cdot (n-3)$$ $$ES-G''_{k} = 0 \quad \mathrm{for} \quad k > 5$$


In the following, we print $S-G'_{i}$ / $ES-G'_{i+1}$ or $S-G''_{i}$ / $ES-G''_{i+1}$, so don't forget that $S-G''_{i} \ne S-G_{i} \ne S-G'_{i} $ in general.

In [11]:
for func, n, string in zip(funcs, n_nodes, strings):
    path = Path("data/temp/test/simple/" + func.__name__)
    flag_path, matrix_path, count_path = func(path, n)
    
    matrix = load_sparse_matrix_from_pkl(matrix_path)
    matrix_info, links = prepare_shared_memory(matrix, 'f')
    bid_matrix = matrix.multiply(matrix.T)
    bid_matrix_info, bid_links = prepare_shared_memory(bid_matrix, 'c')
    
    c_file = h5py.File(count_path, 'r')
    time.sleep(2)
    print(string + ' on ' + str(n))
    
    print("n: Extended simplices on n nodes/ simplices on n-1 nodes")
    for key in c_file.keys():
        simplices = c_file[key]
        mp_iterator = product(simplices, [matrix_info], [bid_matrix_info])
        
        count = 0
        
        r = pool.imap(get_n_extended_simplices, mp_iterator)
        for result in r:
            count += result[0]
        print(str(int(key[-1])+2) + ": " + str(count) + " / " + str(len(simplices)))
        
    for link in links + bid_links:
        link.unlink()

/home/msantoro/miniconda3/envs/thesis38/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
100%|██████████| 100/100 [00:00<00:00, 376508.44it/s]
150it [00:00, 170824.22it/s]
100%|██████████| 51/51 [00:00<00:00, 124655.89it/s]
150it [00:00, 212692.90it/s]

Cyclic graph extension with n nodes on 50
n: Extended simplices on n nodes/ simplices on n-1 nodes
3: 50 / 150



100%|██████████| 101/101 [00:00<00:00, 781595.39it/s]
300it [00:00, 487520.81it/s]

Cyclic graph extension with 1 node on 50
n: Extended simplices on n nodes/ simplices on n-1 nodes
3: 2500 / 150
4: 2400 / 150



100%|██████████| 20/20 [00:00<00:00, 207126.12it/s]
65it [00:00, 290960.26it/s]

...

100%|██████████| 11/11 [00:00<00:00, 129599.28it/s]
40it [00:00, 125390.25it/s]

7: 210 / 210
8: 120 / 120
9: 45 / 45
10: 10 / 10
11: 1 / 1



100%|██████████| 21/21 [00:00<00:00, 193583.26it/s]
80it [00:00, 324825.09it/s]

Circulant graph d. 2 extension with one node on 10
n: Extended simplices on n nodes/ simplices on n-1 nodes
3: 110 / 40
4: 170 / 70
5: 70 / 40


Circulant graph d. 2 extension with one node on 20
n: Extended simplices on n nodes/ simplices on n-1 nodes
3: 420 / 80
4: 740 / 140
5: 340 / 80


We denote by $BS-G_{i}$ the number of bisimplices on $i$ nodes in graph $G$. Then $BS-G'_{i} = 0$ for any graph with no bidirectional connections. 

If $G$ is the cyclic graph on $n$ nodes, then $$BS-G''_{3} = n$$ $$BS-G''_{k} = 0 \quad \mathrm{for} \quad k>3$$
If $G$ is the circulant graph of order $2$ on $n$ nodes, then $$BS-G''_{3} = 2n$$ $$BS-G''_{4} = n$$ $$BS-G''_{k} = 0 \quad \mathrm{for} \quad k>4$$

In [12]:
for func, n, string in zip(funcs, n_nodes, strings):
    path = Path("data/temp/test/simple/" + func.__name__)
    flag_path, matrix_path, count_path = func(path, n)
    
    matrix = load_sparse_matrix_from_pkl(matrix_path)
    matrix_info, links = prepare_shared_memory(matrix, 'f')
    bid_matrix = matrix.multiply(matrix.T)
    bid_matrix_info, bid_links = prepare_shared_memory(bid_matrix, 'c')
    
    c_file = h5py.File(count_path, 'r')
    time.sleep(2)
    
    print(string + ' on ' + str(n))
    
    print("n: Bisimplices on n nodes/ simplices on n-1 nodes")
    
    for key in c_file.keys():
        simplices = c_file[key]
        mp_iterator = product(simplices, [matrix_info], [bid_matrix_info])
        
        count = set()
        
        r = pool.imap(get_bisimplices, mp_iterator)
        for result in r:
            count = count.union(result)
            
        print(str(int(key[-1])+2) + ": " + str(len(count)) + " / " + str(len(simplices)))
        
    for link in links + bid_links:
        link.unlink()

100%|██████████| 100/100 [00:00<00:00, 687590.82it/s]
150it [00:00, 308707.36it/s]
100%|██████████| 51/51 [00:00<00:00, 471166.31it/s]
150it [00:00, 455572.48it/s]

Cyclic graph extension with n nodes on 50
n: Bisimplices on n nodes/ simplices on n-1 nodes
3: 0 / 150


Cyclic graph extension with 1 node on 50
n: Bisimplices on n nodes/ simplices on n-1 nodes
3: 50 / 150


100%|██████████| 101/101 [00:00<00:00, 675637.49it/s]
300it [00:00, 432997.66it/s]

4: 0 / 150


...

100%|██████████| 11/11 [00:00<00:00, 134511.21it/s]
40it [00:00, 205855.41it/s]

6: 0 / 252
7: 0 / 210
8: 0 / 120
9: 0 / 45
10: 0 / 10
11: 0 / 1



100%|██████████| 21/21 [00:00<00:00, 181235.36it/s]
80it [00:00, 312424.88it/s]

Circulant graph d. 2 extension with one node on 10
n: Bisimplices on n nodes/ simplices on n-1 nodes
3: 20 / 40
4: 10 / 70
5: 0 / 40


Circulant graph d. 2 extension with one node on 20
n: Bisimplices on n nodes/ simplices on n-1 nodes
3: 40 / 80
4: 20 / 140
5: 0 / 80
